**환경설정**

In [ ]:
## setup(1min)
# torch 버전 다운. torch>=2.0 에선 colosalai가 동작안함
!pip uninstall torch -y
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

import torch

# for transformers, 최신버전은 에러발생
!pip install transformers==4.28.1

# for ColossalAI
!pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1

**SFT**

In [2]:
# import
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field

def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa

In [8]:
# Define argument

parser = argparse.ArgumentParser()
parser.add_argument("--data_path_1_SFT", type=str, default="./data_kochatgpt/kochatgpt_1_SFT.jsonl")
parser.add_argument("--data_path_1_SFT_add", type=str, default='./data_kochatgpt/instruction_prompt.json')
parser.add_argument("--model_name", type=str, default='skt/kogpt2-base-v2')
parser.add_argument("--max_epochs", type=int, default=2)
parser.add_argument("--train_batch_size", type=int, default=8)
parser.add_argument("--output_dir", type=str, default='./output_1_SFT')

args = parser.parse_args(args=[])

print(args)

Namespace(data_path_1_SFT='./data_kochatgpt/kochatgpt_1_SFT.jsonl', data_path_1_SFT_add='./data_kochatgpt/instruction_prompt.json', model_name='skt/kogpt2-base-v2', max_epochs=2, train_batch_size=8, output_dir='./output_1_SFT')


In [4]:
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{prompt}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    ),
}

In [38]:
# Prepare data
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):
  def __init__(self, data_path_1_SFT: str, data_path_1_SFT_add:str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
    super(SFT_dataset, self).__init__()
    logging.warning("Loading data...")

    # Format
    pattern_instruction = 'prompt'
    pattern_input = "input"
    pattern_output = 'completion'

    # Load dataset
    with open(data_path_1_SFT, "r", encoding="utf-8-sig") as json_file:
      list_data_dict = json.load(json_file)
      if verbose:
        print((list_data_dict[0]))

    with open(data_path_1_SFT_add, "r", encoding="utf-8-sig") as json_file:
      list_data_dict_1 = json.load(json_file)
      if verbose:
        print((list_data_dict_1[0]))

    # list_data_dict.append(list_data_dict_1)
    # print((list_data_dict[-1]))

    list_data_dict += list_data_dict_1

    # 데이터셋 만들기
    prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]

    # 입력
    sources = []
    for example in list_data_dict:
      if example.get(pattern_input, "") != "":
        tmp = prompt_input.formap_map(example)
      else:
        tmp = prompt_no_input.format_map(example)
      sources.append(tmp)

    # 출력
    targets = []
    for example in list_data_dict:
      targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")

    if verbose:
      idx = 0
      print((sources[idx]))
      print((targets[idx]))
      print("Tokenizing inputs... This may take some time")

    # 입력 + 출력 = examples
    examples = [s + t for s, t in zip(sources, targets)]

    # data tokenized
    sources_tokenized = self._tokenize_fn(sources, tokenizer)
    examples_tokenized = self._tokenize_fn(examples, tokenizer)

    # 학습은 target 부분만
    input_ids = examples_tokenized['input_ids']
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized['input_ids_lens']):
      label[:source_len] = IGNORE_INDEX

    data_dict = dict(input_ids = input_ids, labels=labels)

    self.input_ids = data_dict['input_ids']
    self.labels = data_dict['labels']

  def _tokenize_fn(self, strings:Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    tokenized_list = [
         tokenizer(
            text,
            return_tensors = "pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=False,
         )
         for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
         input_ids = input_ids,
         labels=labels,
         input_ids_lens = input_ids_lens,
         labels_lens = labels_lens,
    )
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, i) -> Dict[str, torch.Tensor]:
    return dict(input_ids=self.input_ids[i], labels=self.labels[i])

@dataclass
class DataCollatorForSupervisedDataset(object):
  tokenizer: transformers.PreTrainedTokenizer

  def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
    input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
    )
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
    return dict(
        input_ids=input_ids,
        labels=labels,
        attention_mask = input_ids.ne(self.tokenizer.pad_token_id),
    )

from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token="</s>", eos_token='</s>',
                                                    pad_token='<pad>', mask_token='<mask>')
train_dataset = SFT_dataset(data_path_1_SFT=args.data_path_1_SFT, data_path_1_SFT_add=args.data_path_1_SFT_add, tokenizer=tokenizer)
eval_dataset = None
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [31]:
# 모델 준비
model = AutoModelForCausalLM.from_pretrained(args.model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right",
    model_max_length=512,
)
tokenizer.add_special_tokens(
    {
        'eos_token': DEFAULT_EOS_TOKEN,
        'bos_token': DEFAULT_BOS_TOKEN,
        'unk_token': DEFAULT_UNK_TOKEN,
    }
)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 13.5 MB/s eta 0:00:00


In [39]:
# 학습
training_args = TrainingArguments(
    output_dir="./test",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_steps=3,
    save_steps=1000,
    warmup_steps=5,
    prediction_loss_only=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()
trainer.save_state()
safe_save_model_for_hf_trainer(trainer=trainer, output_dir=args.output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.298000
1000,3.179900
1500,3.076700
2000,3.091400
2500,3.031800
3000,2.984900
3500,2.935400
4000,2.913700
4500,2.898200
5000,2.896000


In [50]:
# 테스트
generator = pipeline('text-generation', model=args.output_dir, tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375,
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

In [48]:
def recom_chat(input: str):
  list_prompt = [input]
  list_prompt = [PROMPT_DICT['prompt_no_input'].format_map({"prompt": tmp}) for tmp in list_prompt]

  list_result = generator(list_prompt, **generation_args)

  for prompt, result in zip(list_prompt, list_result):
    print(("#" * 70))
    print(("completion: %s"%(result[0]['generated_text'])))

In [51]:
recom_chat("초등학생이 읽을 만한 책 추천해 줘")

######################################################################
completion: Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
초등학생이 읽을 만한 책 추천해 줘

### Response(응답):'1. "아내의 유혹" (허영만 저)
- 이 책은 허영만 작가의 베스트셀러 중 하나입니다. 허영만 작가가 어린 시절에 겪은 어려움과 성장 과정을 통해 그의 인생에 대한 이야기를 풀어냅니다.




In [52]:
recom_chat("수학 공부하려고 하는데 무슨 책이 좋을까?")

######################################################################
completion: Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
수학 공부하려고 하는데 무슨 책이 좋을까?

### Response(응답):'1. "수학 공부의 기본 원칙" (윤영철 저)
- 이 책은 수학의 기본 개념과 원리를 이해하는 데 도움을 줄 수 있는 책입니다. 수학 공부는 수학적 사고력을 향상시키는 데 중요한 역할을 합니다.\n\n2. "수학 강의" (오종열 저)
- 윤영철


In [53]:
recom_chat("정세랑 작가 책 추천해 줘")

######################################################################
completion: Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
정세랑 작가 책 추천해 줘

### Response(응답):'저는 AI 어시스턴트이기 때문에 정세랑 작가의 책을 추천해 드릴 수는 없습니다. 하지만 인터넷 검색을 통해 쉽게 찾아보실 수 있으니 참고하시면 좋을 것 같습니다. 감사합니다! 제가 추천하는 책은 다음과 같습니다:



In [54]:
recom_chat("요새 가장 인기 많은 책이 뭐야?")

######################################################################
completion: Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
요새 가장 인기 많은 책이 뭐야?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 정확한 답변을 제공할 수 없습니다. 하지만 인터넷 검색을 통해 최신 정보를 찾아보실 수 있을 것입니다. "가장 인기 있는 책"은 다음과 같습니다:



In [55]:
recom_chat("로맨스 소설 추천해 줘")

######################################################################
completion: Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
로맨스 소설 추천해 줘

### Response(응답):'저는 인공지능 어시스턴트이기 때문에, 로맨스 소설 추천을 해드릴 수 없습니다. 하지만, 로맨스 소설은 다양한 독자층을 가진 베스트셀러 중 하나입니다. 대표작으로는 다음과 같은 것들이 있습니다.\n\n1. "오즈의 마법사" (J.R. 톨킨 저)
- 이 소설은 마
